In [4]:
import os
import sys
import math
import numpy as np
import netCDF4 as nc
from matplotlib import pyplot as plt
from math import exp, sqrt, log
import time
import geopandas as gpd
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
if '..' not in sys.path:
    sys.path.append('..')
from importlib import reload
#reload(util_tools)
import util_tools
from util_tools.data_loader import data_processer

C:\Users\96349\anaconda3\envs\Downscale_env\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\96349\anaconda3\envs\Downscale_env\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\96349\anaconda3\envs\Downscale_env\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\96349\anaconda3\env

# 1. Data Processing

In [2]:
# define necessary parameters
target_var = 'DUSMASS'
file_path_g_06 = r'C:\Users\96349\Documents\Downscale_data\MERRA2\G5NR_aerosol_variables_over_MiddleEast_daily_20060516-20070515.nc'
file_path_g_05 = r'C:\Users\96349\Documents\Downscale_data\MERRA2\G5NR_aerosol_variables_over_MiddleEast_daily_20050516-20060515.nc'
file_path_m = r'C:\Users\96349\Documents\Downscale_data\MERRA2\MERRA2_aerosol_variables_over_MiddleEast_daily_20000516-20180515.nc'
file_path_ele = r'C:\Users\96349\Documents\Downscale_data\elevation\elevation_data.npy'
file_path_country = [r'C:\Users\96349\Documents\Downscale_data\Country_shape\AFG_adm/AFG_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\ARE_adm/ARE_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\IRQ_adm/IRQ_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\KWT_adm/KWT_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\QAT_adm/QAT_adm0.shp',
                     r'C:\Users\96349\Documents\Downscale_data\Country_shape\SAU_adm/SAU_adm0.shp']
n_lag = 4
n_pred = 2
task_dim = [5, 5]

In [188]:
reload(util_tools.data_loader)
import util_tools.data_loader as data_loader

In [189]:
data_processor = data_loader.data_processer()
g_data, m_data, [G_lats, G_lons, M_lats, M_lons], ele_data = data_processor.load_data(target_var, file_path_g_05, file_path_g_06, file_path_m, file_path_ele, file_path_country)

In [190]:
m_data = data_processor.unify_m_data(g_data[:10], m_data, G_lats, G_lons, M_lats, M_lons)

In [191]:
match_m_data = m_data[range(1826, 1826+730), :, :]

In [9]:
os.path.join('a', 'test.npy')

'a\\test.npy'

In [192]:
X_high, X_low, X_ele, X_other, Y = data_processor.flatten(g_data[:10, :10, :10], match_m_data[:10, :10, :10], ele_data[:10, :10],
                                                          [G_lats[:10], G_lons[:10]], list(range(10)), n_lag=n_lag, n_pred=n_pred, task_dim=task_dim, is_perm=False, 
                                                          return_Y=True)

In [146]:
def reconstruct(pred_Y, org_dim, task_dim):
    rec_Y = dict()
    lat_dim = org_dim[0]+1 - pred_Y.shape[-2]
    lon_dim = org_dim[1]+1 -pred_Y.shape[-1]
    for i, lat_corner in enumerate(range(pred_Y.shape[-2], org_dim[0]+1), 1):
        for j, lon_corner in enumerate(range(pred_Y.shape[-1], org_dim[1]+1), 1):
            current_index = (i-1)*lon_dim + j-1
            current_mtx = pred_Y[current_index]
            for k, lat in enumerate(range(lat_corner-task_dim[0], lat_corner)):
                for h, lon in enumerate(range(lon_corner-task_dim[1], lon_corner)):
                    if (lat, lon) not in rec_Y:
                        rec_Y.setdefault((lat, lon), [current_mtx[k, h]])
                    else:
                        rec_Y[(lat, lon)].append(current_mtx[k, h])
    out = np.zeros(org_dim)
    for a in range(org_dim[0]):
        for b in range(org_dim[1]):
            out[a, b] = np.mean(rec_Y[(a, b)])
    return out

In [147]:
recon_Y = reconstruct(Y[:, 0, :, :], [10, 10], task_dim)

In [155]:
pred_list = [recon_Y, recon_Y, recon_Y]
np.expand_dims(np.array(pred_list), 1).shape

(3, 1, 10, 10)

In [179]:
temp_matrix = np.zeros((3,3,10, 10))
pred_list = [recon_Y,  recon_Y, recon_Y]
temp_matrix = np.concatenate([temp_matrix[1:], np.expand_dims(np.array(pred_list), 0)], axis=0)

In [12]:
import time 
time.time()

1663883632.2123516

# 2. GAN model test

In [193]:
def nnelu(input):
    return tf.add(tf.constant(1, dtype=tf.float32), tf.nn.elu(input))

high_input = tf.keras.Input(shape=(n_lag, task_dim[0], task_dim[1], 1))
x1 = tf.keras.layers.ConvLSTM2D(16, kernel_size=(3,3), return_sequences=True, activation=tf.keras.layers.LeakyReLU())(high_input)
x1 = tf.keras.layers.ConvLSTM2D(16, kernel_size=(3,3), return_sequences=True, activation=tf.keras.layers.LeakyReLU())(x1)
x1 = tf.keras.layers.ConvLSTM2D(16, kernel_size=(1,1), activation=tf.keras.layers.LeakyReLU())(x1)
x1 = tf.keras.layers.Flatten()(x1)

low_input = tf.keras.Input(shape=(n_lag, task_dim[0], task_dim[1], 1))
x2 = tf.keras.layers.ConvLSTM2D(16, kernel_size=(3,3), activation=tf.keras.layers.LeakyReLU())(low_input)
x2 = tf.keras.layers.Flatten()(x1)

ele_input = tf.keras.Input(shape=(task_dim[0], task_dim[1], 1))
x3 = tf.keras.layers.Conv2D(16, kernel_size=(3,3), activation=tf.keras.layers.LeakyReLU())(ele_input)
x3 = tf.keras.layers.Flatten()(x3)

other_input =  tf.keras.Input(shape=(3))
x4 = tf.keras.layers.Dense(8, activation=tf.keras.layers.LeakyReLU())(other_input)

x = tf.keras.layers.Concatenate(axis=1)([x1, x2, x3, x4])
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(30, activation=tf.keras.layers.LeakyReLU())(x)
x = tf.keras.layers.Dense(n_pred*np.prod(task_dim), activation=nnelu)(x)
x = tf.keras.layers.Reshape([n_pred, task_dim[0], task_dim[1]])(x)
generator = tf.keras.Model([high_input, low_input, ele_input, other_input], x)
generator.compile(optimizer='adam', loss='mean_absolute_error')

In [194]:
generator.fit([X_high, X_low, X_ele, X_other], Y, epochs=100)

Epoch 1/100
6/6 [==============================] - 10s 15ms/step - loss: 0.2156
Epoch 2/100
6/6 [==============================] - 0s 16ms/step - loss: 0.1636
Epoch 3/100
6/6 [==============================] - 0s 16ms/step - loss: 0.1244
Epoch 4/100
6/6 [==============================] - 0s 17ms/step - loss: 0.1007
Epoch 5/100
6/6 [==============================] - 0s 17ms/step - loss: 0.0839
Epoch 6/100
6/6 [==============================] - 0s 18ms/step - loss: 0.0712
Epoch 7/100
6/6 [==============================] - 0s 18ms/step - loss: 0.0621
Epoch 8/100
6/6 [==============================] - 0s 19ms/step - loss: 0.0526
Epoch 9/100
6/6 [==============================] - 0s 19ms/step - loss: 0.0450
Epoch 10/100
6/6 [==============================] - 0s 18ms/step - loss: 0.0404
Epoch 11/100
6/6 [==============================] - 0s 19ms/step - loss: 0.0362
Epoch 12/100
6/6 [==============================] - 0s 18ms/step - loss: 0.0324
Epoch 13/100
6/6 [==============================

In [195]:
pred_Y = generator.predict([X_high[-1:], X_low[-1:], X_ele[-1:], X_other[-1:]])

In [196]:
pred_Y.shape

(1, 2, 5, 5)

In [112]:
np.sqrt(np.mean(np.square(pred_Y[-1] - Y[-1])))

0.010288388300887755

In [100]:
from util_tools.data_processing import rsquared
rsquared(pred_Y[0, 0].reshape((25)), Y[0,0].reshape((25)))

(0.008515162367835552, 0.6608753429840621)

In [117]:
pred_input = tf.keras.Input(shape=(n_pred, task_dim[0], task_dim[1]))
y1 = tf.keras.layers.Flatten()(pred_input)
'''
condition_input = tf.keras.Input(shape=(3))
y2 = tf.keras.layers.Dense(8, activation='relu')(condition_input)
y = tf.keras.layers.Concatenate(axis=1)([y1, y2])
'''
y = tf.keras.layers.Dense(8, activation=tf.keras.layers.LeakyReLU())(y1)
y = tf.keras.layers.Dropout(0.8)(y)
y = tf.keras.layers.Dense(1, activation='sigmoid')(y)
discriminator = tf.keras.Model([pred_input], y)
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [118]:
reload(util_tools.cGAN_model)
from util_tools.cGAN_model import  Condition_GAN

In [122]:
cGAN = Condition_GAN(generator, discriminator)
cGAN.fit(5, 30, [X_high, X_low, X_ele, X_other], Y)

Epoch:1, batch:1/6, real_loss=0.634, fake_loss=0.769, g_loss=0.692
Epoch:1, batch:2/6, real_loss=0.586, fake_loss=0.953, g_loss=0.705
Epoch:1, batch:3/6, real_loss=0.691, fake_loss=0.595, g_loss=0.688
Epoch:1, batch:4/6, real_loss=0.383, fake_loss=0.807, g_loss=0.591
Epoch:1, batch:5/6, real_loss=0.594, fake_loss=0.430, g_loss=0.600
Epoch:1, batch:6/6, real_loss=0.358, fake_loss=0.644, g_loss=0.589
Epoch:2, batch:1/6, real_loss=0.527, fake_loss=0.627, g_loss=0.709
Epoch:2, batch:2/6, real_loss=0.447, fake_loss=0.860, g_loss=0.680
Epoch:2, batch:3/6, real_loss=0.383, fake_loss=0.463, g_loss=0.615
Epoch:2, batch:4/6, real_loss=0.484, fake_loss=0.496, g_loss=0.686
Epoch:2, batch:5/6, real_loss=0.562, fake_loss=0.471, g_loss=0.647
Epoch:2, batch:6/6, real_loss=0.605, fake_loss=0.767, g_loss=0.621
Epoch:3, batch:1/6, real_loss=0.603, fake_loss=0.329, g_loss=0.619
Epoch:3, batch:2/6, real_loss=0.694, fake_loss=0.643, g_loss=0.605
Epoch:3, batch:3/6, real_loss=0.424, fake_loss=0.661, g_loss=0

In [123]:
generator.predict([X_high[:3], X_low[:3], X_ele[:3], X_other[:3]])

array([[[[0.8665214 , 1.2154813 , 0.5742026 , 1.1931497 , 0.71439743],
         [0.9954182 , 0.3683976 , 0.8625089 , 1.425251  , 0.50635064],
         [0.5054725 , 0.9580225 , 1.0433807 , 0.6285501 , 1.0431772 ],
         [0.7981943 , 1.5731097 , 0.9249619 , 0.8616722 , 0.59727335],
         [0.5140573 , 0.95949286, 0.6270662 , 1.241638  , 0.75372356]],

        [[1.5097042 , 1.2523549 , 0.48150563, 1.2555624 , 0.7489146 ],
         [1.6177773 , 0.7694826 , 0.40843403, 0.9733394 , 0.8524309 ],
         [1.186528  , 1.5459706 , 1.0745908 , 1.2855966 , 1.4296491 ],
         [0.7716894 , 0.9346115 , 0.77756846, 1.240102  , 0.8940858 ],
         [0.54506534, 1.1562147 , 1.3063023 , 1.5097346 , 0.72588825]]],


       [[[0.86630523, 1.2159221 , 0.5721369 , 1.1952002 , 0.71524763],
         [0.9942844 , 0.36784196, 0.8628505 , 1.4258636 , 0.50646466],
         [0.5058225 , 0.9570836 , 1.043303  , 0.62991464, 1.0408068 ],
         [0.7978578 , 1.5721087 , 0.92737603, 0.8630987 , 0.59764117],


In [25]:
X = [X_high, X_low, X_ele, X_other]
j =0
batch_size=2
batch_X = [d[j*batch_size:(j+1)*batch_size] for d in X]

In [28]:
batch_X[0].shape

(2, 4, 5, 5, 1)

In [30]:
np.zeros((batch_size, 1), dtype='int')

array([[0],
       [0]])

# 3. downscaler test

In [215]:
reload(downscale)
from util_tools import downscale

In [216]:
dscler = downscale.downscaler(generator)

In [217]:
h_data = g_data[10:10+n_lag]
l_data = m_data[1836: 1856]
days = list(range(1836, 1856))

In [219]:
downscaled_data = dscler.downscale(h_data, l_data, ele_data,  [G_lats, G_lons, M_lats, M_lons], days, n_lag, n_pred, task_dim)

In [220]:
downscaled_data.shape

(16, 123, 207)

In [221]:
h_data.shape

(4, 123, 207)